In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
# Sample file, from Dec 2024
# https://mbta-massdot.opendata.arcgis.com/datasets/924df13d845f4907bb6a6c3ed380d57a/about
filename = "MBTA-Bus-Arrival-Departure-Times_2024-12.csv"


In [ ]:
df = pd.read_csv(filename, low_memory=False)

In [ ]:
print(df.head())
print(df.shape)
print(df.describe)

In [ ]:
# Calculate delay for each station in seconds
df['time_difference'] = pd.to_datetime(df['actual']) - pd.to_datetime(df['scheduled'])
df['delay_seconds'] = df['time_difference'].dt.total_seconds()


In [ ]:
# Calculate headway delay for each station in seconds 
# Headway is the amount of time that has passed since the previous bus stopped at the station
df['headway_delay'] = df['headway'] - df['scheduled_headway']

In [ ]:
print(df.head())


In [ ]:
# Save as a CSV if wanted, optional to check manually
df.to_csv('MBTA-Bus-Arrival-Departure-Times_2024-12-with-delay.csv', index=False)